The Domain

The main goal is to generate a Matchcard. Sample Matchcards can be found in the Excel Sheet. A Team's Matchcard contains these informations:
* The Treasury of the team before a game
* The Winnings of the team in the match
* The upkeep, which is depending on whether or not players are Starplayers and how high the initial cost of the starplayer was
* The future treasury, which is the treasury before the game + winnings - upkeep
* Also, in case the coach has players with special roles, special bowlbot commands are generated.

In [1]:
type Currency = Currency of int

module Currency =
    let sum cs = 
        Currency(List.sumBy (fun (Currency c) -> c) cs)

type Position =
    | Shaman
    | PlayerCoach
    | Cheerleader
    | Filcher
    | BribedRef
    | Fan
    | SquigTamer

module Position =
    let convert s = 
        match s with
        | "Shaman" -> Some Shaman
        | "Player-Coach" -> Some PlayerCoach
        | "Cheerleader" -> Some Cheerleader
        | "Filcher" -> Some Filcher
        | "Bribed Ref" -> Some BribedRef
        | "Fan" -> Some Fan
        | "Squig Tamer" -> Some SquigTamer
        | _ -> None

    [<Literal>] 
    let BB1D6 = "bb1d6"

    let toBowlbotMessage p =
        match p with
        | Shaman -> $"{BB1D6},1d8"
        | PlayerCoach -> "bb1dX,1d15"
        | Cheerleader -> BB1D6
        | Filcher -> BB1D6
        | BribedRef -> BB1D6
        | Fan -> BB1D6
        | SquigTamer -> BB1D6

    let toEmote p =
        match p with
        | Shaman -> ":shaman:"
        | PlayerCoach -> ":coach:"
        | Cheerleader -> ":cheer:"
        | Filcher -> ":filcher:"
        | BribedRef -> ":ref:"
        | Fan -> ":fan:"
        | SquigTamer -> ":tamer:"

type PositionData = {
    Cost : Currency
}

module PositionData = 
    let create cost = {
        Cost = cost
    }

    let costToUpkeep (Currency c) =
        if c < 56000 then (Currency 10000)
        elif c < 106000 then (Currency 20000)
        elif c < 156000 then (Currency 30000)
        elif c < 206000 then (Currency 40000)
        elif c < 301000 then (Currency 50000)
        else (Currency 60000)

type Roster = {
    Positions : Map<int, PositionData>
}

module Roster =
    let create positions = {
        Positions = positions
    }

type Player = 
    | Normal of string
    | Star of string * Currency
    | Special of string * Position

type EmoteProvider =
    | Position of Position
    | Winnings
    | Upkeep

module EmoteProvider =
    let toEmote ep =
        match ep with
        | Position p -> Position.toEmote p
        | Winnings -> ":winnings:"
        | Upkeep -> ":upkeep:"

type FanFactor = FanFactor of int

type Team = {
    Name : string
    Treasury : Currency
    FanFactor : FanFactor
    Players : Player list
}

module Team =
    let create name treasury fanfactor players roster = {
        Name = name
        Treasury = treasury
        FanFactor = fanfactor
        Players =
            players 
            |> List.map (fun (playerName, playerPosition, positionId) -> 
                if playerPosition.Equals(playerName) then
                    let { Cost = cost } = 
                        match roster.Positions.TryFind positionId with
                        | Some data -> data
                        | None -> failwith $"Couldnt find Position {positionId} in Team {name}"
                    Star (playerName, PositionData.costToUpkeep cost)
                else
                    match Position.convert playerPosition with
                    | Some pos -> Special (playerName, pos)
                    | None -> Normal playerName
                )
    }

    let getUpkeep t =
        t.Players 
            |> List.choose (fun p -> 
                match p with
                | Star (_, upkeep) -> Some upkeep
                | _ -> None)
            |> Currency.sum

type MatchResult = {
    Winnings : Currency
}

module MatchResult =
    let create winnings = {
        Winnings = winnings
    }

type Match = {
    Team1 : Team * MatchResult
    Team2 : Team * MatchResult
}

module Match =
    let create team1 team2 = { 
        Team1 = team1
        Team2 = team2
    }

type MatchCard = {
    Team : Team
    Result : MatchResult
}

module Matchcard =
    let create team result = {
        Team = team
        Result = result
    }

    let positionToMessage team position =
        let rec getMessage position players =
            match players with
            | [] -> "Not rostered"
            | head::tail ->
                match head with
                | Special (_,p) when p = position -> Position.toBowlbotMessage p
                | _ -> getMessage position tail
        $"{EmoteProvider.toEmote (Position position)} {getMessage position team.Players}"

    let treasuryBeforeMatchToMessage (Currency current) (Currency winnings) =
        $"Treasury: {current-winnings}"

    let futureTreasuryToMessage (Currency current) (Currency upkeep) =
        $"Future Treasury: {current-upkeep}"

    let winningsToMessage (Currency winnings) =
        $"{EmoteProvider.toEmote Winnings} {winnings}"

    let upkeepToMessage (Currency upkeep) =
        $"{EmoteProvider.toEmote Upkeep} {upkeep}"

    let print mc delimiter =
        let upkeep = Team.getUpkeep mc.Team
        let p2m = positionToMessage mc.Team

        let addMessage m text =
            text + m + delimiter
            
        String.Empty
        |> addMessage mc.Team.Name
        |> addMessage "XXXLBowl #1"
        |> addMessage (treasuryBeforeMatchToMessage mc.Team.Treasury mc.Result.Winnings)
        |> addMessage (winningsToMessage mc.Result.Winnings)
        |> addMessage (p2m BribedRef)
        |> addMessage (p2m Fan)
        |> addMessage (upkeepToMessage upkeep)
        |> addMessage (futureTreasuryToMessage mc.Team.Treasury upkeep)
        |> addMessage "Specialist Rolls:"
        |> addMessage (p2m Cheerleader)
        |> addMessage (p2m Fan)
        |> addMessage (p2m Filcher)
        |> addMessage (p2m PlayerCoach)
        |> addMessage (p2m Shaman)
        |> addMessage (p2m SquigTamer)

    let printTeam t delimiter =
        let (team, result) = t
        let mc = create team result
        print mc delimiter

    let printMatch m delimiter newLine =
        (printTeam m.Team1 delimiter) + newLine + (printTeam m.Team2 delimiter)

The API
https://fumbbl.com/apidoc/#

In [1]:
#r "nuget: FsHttp"

open FsHttp
open FSharp.Data

[<Literal>] 
let FumblAPI = "https://fumbbl.com/api/"

module RosterAPI =
    [<Literal>] 
    let Endpoint = FumblAPI + "roster/get/"
    [<Literal>] 
    let Sample = Endpoint + "4950"
    type Data = JsonProvider<Sample>

    let get (i:int) =
        http {
            GET $"{Endpoint}{i}"
        }
        |> toText
        |> Data.Parse
        |> fun r ->
            let positions = 
                r.Positions
                |> Array.toList
                |> List.map (fun p -> (p.Id, PositionData.create (Currency p.Cost)))
                |> Map.ofList
            Roster.create positions

module TeamAPI =
    [<Literal>] 
    let Endpoint = FumblAPI + "team/get/"
    [<Literal>] 
    let Sample = Endpoint + "997349"
    type Data = JsonProvider<Sample>

    let get (i:int) =
        http {
            GET $"{Endpoint}{i}"
        }
        |> toText
        |> Data.Parse
        |> fun t -> 
            let name = t.Name
            let treasury = Currency t.Treasury
            let fanFactor = FanFactor t.FanFactor
            let players = 
                t.Players 
                |> Array.toList 
                |> List.map (fun p -> (p.Name, p.Position, p.PositionId))
            let roster = RosterAPI.get t.Roster.Id
            Team.create name treasury fanFactor players roster

module MatchAPI =
    [<Literal>] 
    let Endpoint = FumblAPI + "match/get/"
    [<Literal>] 
    let Sample = Endpoint + "4284928"
    type Data = JsonProvider<Sample>

    let get (i:int) = 
        http {
            GET $"{Endpoint}{i}"
        }
        |> toText
        |> Data.Parse
        |> fun m -> 
            let team1 = TeamAPI.get m.Team1.Id
            let result1 = MatchResult.create (Currency m.Team1.Winnings)
            let team2 = TeamAPI.get m.Team2.Id
            let result2 = MatchResult.create (Currency m.Team2.Winnings)
            Match.create (team1, result1) (team2, result2)

To get the required data to create MatchCards for a Match first, load the match with 
```
MatchAPI.get {MatchID}
```

In [1]:
let matchData = MatchAPI.get 4284928

Then create and ouput the Matchcard Data with
```
Matchcard.printMatch {matchData} {delimiter} {newLine}
```
We use NewLine as the delimiter which we get with
```
System.Environment.NewLine
```

In [1]:
let newLine = System.Environment.NewLine
let delimiter = newLine

In [1]:
Matchcard.printMatch matchData delimiter newLine

Bogan Picnic Hamper Stealers
XXXLBowl #1
Treasury: 144000
:winnings: 60000
:ref: Not rostered
:fan: Not rostered
:upkeep: 0
Future Treasury: 204000
Specialist Rolls:
:cheer: Not rostered
:fan: Not rostered
:filcher: bb1d6
:coach: Not rostered
:shaman: Not rostered
:tamer: Not rostered

Black Tooth Rips
XXXLBowl #1
Treasury: 11000
:winnings: 70000
:ref: bb1d6
:fan: bb1d6
:upkeep: 70000
Future Treasury: 11000
Specialist Rolls:
:cheer: bb1d6
:fan: bb1d6
:filcher: bb1d6
:coach: Not rostered
:shaman: bb1d6,1d8
:tamer: Not rostered
